In [2]:
#general libraries
import pandas as pd
import numpy as np
import getpass

#SQL
import pymysql
from sqlalchemy import create_engine
#plot
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib

#from scipy.stats import normaltest
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import Normalizer, StandardScaler, PowerTransformer

from sklearn.metrics import confusion_matrix, plot_confusion_matrix, cohen_kappa_score, accuracy_score, classification_report

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks


Using matplotlib backend: Qt5Agg


C:\Users\Borja\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 2. Gather the data

In [3]:
password = getpass.getpass()

········


In [4]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/credit_card_classification'
engine = create_engine(connection_string)
engine.execute("USE credit_card_classification")

In [8]:
query="""select * from credit_card_classification.credit_card_data_2"""
data = pd.read_sql_query(query, engine)

Note, 24 rows were dropped by MySQL in the import from the original CSV file to SQL, because some columns were empty (balance & co). We estimate that 24 out of a total of 18000 is not significant.

Next, we make a copy of the original dataframe so that we can use it again later

In [14]:
data2 = data.copy()

# 3. Clean the data

# 4. Explore the data, select features to be used for the model

In [15]:
# We now set the dataframe index to be the customer_number so that it is not used for the model but, 
# we don't lose the information in case we need it afterwards.
data2.set_index('customer_number', inplace=True) 

In [16]:
# Dropping features with VIF >5
data2 = data2.drop(['bank_accounts_open','credit_cards_held','homes_owned','household_size'], axis=1)

#3. dropping in addition to 2. the avg balance
#data2 = data2.drop(['bank_accounts_open','credit_cards_held','homes_owned','household_size','balance'], axis=1)

## Categorical conversion

In [17]:
# Convert categorical ordinal
# ordinal categoricals have an order and we can substitute the values in just one column, if we respect the order
data2['income_level'] = data2['income_level'].map({'Low':0, 'Medium':1, 'High':2})
data2['credit_rating'] = data2['credit_rating'].map({'Low':0, 'Medium':1, 'High':2})

# We convert the categoricals that have only 2 choices the same way:
data2['offer_accepted'] = data2['offer_accepted'].map({'Yes':1, 'No':0})
data2['overdraft_protection'] = data2['overdraft_protection'].map({'Yes':1, 'No':0})
data2['own_your_home'] = data2['own_your_home'].map({'Yes':1, 'No':0})
data2['mailer_type'] = data2['mailer_type'].map({'Postcard':0, 'Letter':1})

In [18]:
#conversion of the remaining categoricals to numbers via getdummies
data3 = data2['reward']
data2 = data2.drop('reward', axis=1)
data3 = pd.get_dummies(data3,drop_first=True) 


We decided to use drop first because the third option of reward is given already when the other 2 are 0.

We now put everything back together


In [19]:
data4 = pd.concat([data2,data3],axis=1)

# Auxiliary functions for the models

In [28]:
def calculate_display_results(model,predic,X_test,y_test):
    # accuracy_score
    # fitted X_test data vs. y_test data (actual answer)
    accuracy =   round(accuracy_score(predic,y_test),2)
    # Kappa score
    kappa =  round(cohen_kappa_score(y_test,predic),2)
    # classification report
    classif_report = classification_report(predic,y_test)
    # compare predictions to actual answers
    conf_mat = confusion_matrix(predic,y_test).T
    return accuracy, kappa, conf_mat,classif_report

def append_to_results(accuracy, kappa, conf_mat,classif_report):
    infos_from_model =[]
    infos_from_model.append(accuracy)
    infos_from_model.append(kappa)
    infos_from_model.append(conf_mat)
    infos_from_model.append(classif_report)
    return infos_from_model

# 5. Create and 6. Apply different models

In [21]:
# X-y split
X = data4.drop(['offer_accepted'], axis=1)
y = data4['offer_accepted']
y.columns = ['offer_accepted']

In [22]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# We will store the values of the results of the different models in a dictionary, 
# so that we can compare them afterwards.
results ={}

# Fixing the dependend feature imbalance through Smote

We will use the Smote method to oversample the "offer accepted" feature to fix the imbalance.

In [24]:
smote = SMOTE()
X_sm, y_sm = smote.fit_sample(X, y)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=100)

# Random forest

In [26]:
rfc = RandomForestClassifier().fit(X_train, y_train)
predictions = rfc.predict(X_test)

In [29]:
# Calculate results
accuracy, kappa, conf_mat,classif_report = calculate_display_results(rfc,predictions,X_test,y_test)
# Add results to result dictionary
inf_from_model = append_to_results(accuracy, kappa, conf_mat,classif_report)

results['rfc']= inf_from_model

In [30]:
results_df = pd.DataFrame()
results_df['parameters']=['Accuracy','Kappa']
for key, value in results.items() :
    print (key, value[0],value[1])
    results_df[key]=[value[0],value[1]]
results_df.set_index('parameters',inplace=True)
results_df=results_df.T
results_df.sort_values(by =['Accuracy', 'Kappa'],inplace=True)
display(results_df)

rfc 0.93 0.86


parameters,Accuracy,Kappa
rfc,0.93,0.86


# THE END 